# Display raw data
Unfiltered Data is stored as JSONL-Files on the device SD-Card per day.
Data can be transferred using `mpremote fs cp :/sd/gps_logs TARGT_DIR` when `/sd` is mounted in boot.py.

In [10]:
import os
import sys
import json
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import os
import sys
import json
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# Path to GPS log files
raw_dir = '/Users/ralf.sigmund/GitHub/mp_m5_fahrtenbuch/gps_logs/gps_logs'
raw_files = os.listdir(raw_dir)
raw_files = [f for f in raw_files if f.endswith('.jsonl')]

# Read all JSONL files and extract location data
all_locations = []

for file in raw_files:
    file_path = os.path.join(raw_dir, file)
    with open(file_path, 'r') as f:
        for line in f:
            try:
                data = json.loads(line)
                if 'latitude' in data and 'longitude' in data:
                    all_locations.append({
                        'lat': data['latitude'],
                        'lon': data['longitude'],
                        'timestamp': data.get('timestamp', ''),
                        'file': file
                    })
                elif 'lat' in data and 'lon' in data:
                    all_locations.append({
                        'lat': data['lat'],
                        'lon': data['lon'],
                        'timestamp': data.get('timestamp', ''),
                        'file': file
                    })
            except json.JSONDecodeError:
                print(f"Error parsing line in {file}")

# Convert to DataFrame for easier handling
df = pd.DataFrame(all_locations)
print(f"DONE {len(df)}")

DONE 22167


In [ ]:

# Create a map centered at the mean location
mean_lat = df['lat'].mean()
mean_lon = df['lon'].mean()
location_map = folium.Map(location=[mean_lat, mean_lon], zoom_start=12)

# Group the data by file
file_groups = df.groupby('file')

# For each file, create a feature group
for file_name, group in file_groups:
    # Create a feature group for this file
    fg = folium.FeatureGroup(name=f"Track: {file_name}", show=True)
    
    # Add a polyline to show the path
    folium.PolyLine(
        group[['lat', 'lon']].values,
        color='blue',
        weight=2,
        opacity=0.7,
        popup=f"File: {file_name}"
    ).add_to(fg)
    
    # Add start and end markers
    start_point = group.iloc[0]
    end_point = group.iloc[-1]
    
    # Start marker (green)
    folium.Marker(
        location=[start_point['lat'], start_point['lon']],
        popup=f"Start of {file_name}<br>Timestamp: {start_point['timestamp']}",
        tooltip=f"Start: {file_name}",
        icon=folium.Icon(color='green')
    ).add_to(fg)
    
    # End marker (red)
    folium.Marker(
        location=[end_point['lat'], end_point['lon']],
        popup=f"End of {file_name}<br>Timestamp: {end_point['timestamp']}",
        tooltip=f"End: {file_name}",
        icon=folium.Icon(color='red')
    ).add_to(fg)
    
    # Add this feature group to the map
    fg.add_to(location_map)

# Add layer control to toggle file visibility
folium.LayerControl().add_to(location_map)

# Display the map
location_map

: 